## We Can Control Brighter-Fatter Effect in ImSim

Author: Michael Wood-Vasey  
Date: 2018-08-24

#### Abstract

The size difference between XX, YY measured moments and PSF XX, YY model moments is well controlled on example ImSim Run 1.2i image when applying a brighter-fatter correction kernel generated from a pair of high-intensity flats.

#### Details
1. Generate Brighter-Fatter Kernel with [Make Brighter-Fatter Notebook](make_brighter_fatter_kernel.ipynb)
   - Uses a pair of high-intensity (80k e-/pix) flats generated by Jim Chiang
2. Test runs from [Test BF Script](../scripts/test_bf.sh)
   - Uses a full focal plane ImSim image generated by Jim Chiang
   - This script generates an ImSim repo with a rerun without brighter-fatter applied and one with brighter-fatter applied.
3. This Notebook then reads in the catalogs and plots the difference between the object XX, YY measured moments and the PSF XX, YY model moments for each object.

#### Notes:
* The steps above were run in total using the following branches:  
  - `obs_base`  branch `tickets/DM-13293`
  - `obs_lsstCam` branch `tickets/DM-15509`
  - `ip_isr` branch `tickets/DM-13293`
  - `cp_pipe` branch `tickets/DM-13293`  to generate the Brighter-Fatter kernel
* I ran the `processCcd.py` processing on `lsst-dev01.ncsa.illinois.edu` at NCSA due to some small hiccups at NERSC getting the `ip_isr` branch compiled against an up-to-date weeklies.  Nothing serious.  Heather Kelly is on the case and this should all be resolved shortly.

#### Acknowledgments:
Significant thanks to
* Merlin Fisher-Levine for adapting and explaining the Brighter-Fatter kernel code to me
* Jim Chiang for generate simulated images and flats.
* The specific plotting code in this Notebook was adapted from code by Merlin Fisher-Levine  
from code by Lauren MacArthur.


In [ ]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
%matplotlib inline

In [ ]:
def plot(mags, diffs, title='', saveFilename=None):
    maxDiff = 12
    nStdev = 3.0

    ptSize = 25
    fontsize = 20
    color = 'b'

    left, width = 0.12, 0.62
    bottom, height = 0.10, 0.85 # had to change height to make it save correctly
    left_h = left + width + 0.03
    bottom_h = bottom + width + 0.02
    rect_scatter = [left, bottom, width, height]
    rect_histy = [left_h, bottom, 0.20, height]

    deltaMin = 0.0

    fig = plt.figure(figsize=(15,10))
    axScatter = plt.axes(rect_scatter)
    if title: plt.title(title, fontsize=25)
    axHisty = plt.axes(rect_histy)

    axScatter.scatter(mags, diffs, s=1.3*ptSize, marker="o", facecolors="b", edgecolors=color, alpha = 0.15)
    axScatter.axhline(0, ls='--', color="0.4")
    
    # Show bin in magnitudes
    mag_bin_size = 0.1  # mag
    mag_bins = np.arange(16, 21, mag_bin_size)
    bin_index = np.digitize(mags, mag_bins)
    

    axScatterY1 = axScatter.get_ylim()[0]
    axScatterY2 = axScatter.get_ylim()[1]
    nyDecimal = int(-1.0*np.around(np.log10(0.05*abs(axScatterY2 - (axScatterY1 - deltaMin))) - 0.5))
    yBinwidth = max(0.5/10**nyDecimal, np.around(0.02*abs(axScatterY2 - (axScatterY1 - deltaMin)), nyDecimal))

    yBins = np.arange((axScatterY1 - deltaMin) - 0.5*yBinwidth, axScatterY2 + 0.55*yBinwidth, yBinwidth)
    axHisty.hist(diffs, bins=yBins, color='b', alpha=0.6, orientation="horizontal", label='test')

    axScatter.set_ylim(-1.*maxDiff,maxDiff)
    axHisty.set_ylim(-1.*maxDiff,maxDiff)
    axHisty.set_xscale("log", nonposy="clip")
    
    axScatter.set_xlabel('mag')
    axScatter.set_ylabel('Percentage difference star - model size: (sqrt(<xx+yy>/2)')
    
    mean = np.mean(diffs)
    std = np.std(diffs)
    std_clip = np.std( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    mean_clip = np.mean ( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    
    spc = 0.04
    plt.text(0.05, 0.95-0*spc, "Star N       = %s"  %len(diffs), ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-1*spc, "stddev      = %.4f" %std,        ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-2*spc, "std_clip     = %.4f"%std_clip,   ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-3*spc, "mean        = %.4f" %mean,       ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-4*spc, "mean_clip = %.4f"   %mean_clip,  ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')

    if saveFilename:
        plt.savefig(saveFilename)


In [ ]:
from copy import copy

def get_shape_mag_diff(visits, partial_data_id):
    star_sizes = []
    model_sizes = []
    mags = []
    for visit in visits:
        data_id = copy(partial_data_id)
        data_id['visit'] = visit
        try:
            cat = butler.get('src', dataId=data_id)  
            calib = butler.get('calexp_calib', dataId=data_id)
        except dafPersist.NoResults as e:
            print(e)
            continue

        calib.setThrowOnNegativeFlux(False)

        mask = (cat['calib_psfUsed'] == 1) #& (cat['base_PixelFlags_flag_interpolated'] == 0)
        cat = cat[mask]
        star_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_xx'] + cat['base_SdssShape_yy']))])
        model_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_psf_xx'] + cat['base_SdssShape_psf_yy']))])
        mags.extend([_ for _ in calib.getMagnitude(cat['base_PsfFlux_flux'])])
        
    diffs = [100*(a - b)/b for a,b in zip(star_sizes, model_sizes)]
    diffs = np.asarray(diffs)

    return mags, diffs



In [ ]:
outdir = './plots'

try:
    os.makedirs(outdir)
except:
    pass

------
#### No Brighter-Fatter Correction Applied

In [ ]:
repo = '/global/homes/w/wmwv/DC2-production/Notebooks/bf_kernel/imsim_repo/rerun/no_bf_correction'

In [ ]:
run = os.path.basename(repo)
print(run)

In [ ]:
butler = dafPersist.Butler(repo)

In [ ]:
visits = [204595]
detectors = range(189)

In [ ]:
mags = {}
diffs = {}

for detector in detectors:
    partial_data_id = {'detector': detector}
    mags[detector], diffs[detector] = get_shape_mag_diff(visits, partial_data_id)

In [ ]:
# Plot all combined measurements
all_mags = []
all_diffs = []
for detector in detectors:
    all_mags.extend(mags[detector])
    all_diffs.extend(diffs[detector])

title = '{}'.format(run, detector)
saveName = os.path.join(outdir, title + '.pdf')
plot(all_mags, all_diffs, title, saveName)

------
#### With Brighter-Fatter Correction Applied

In [ ]:
repo = '/global/homes/w/wmwv/DC2-production/Notebooks/bf_kernel/imsim_repo/rerun/with_bf_correction'

In [ ]:
run = os.path.basename(repo)
print(run)

In [ ]:
butler = dafPersist.Butler(repo)

In [ ]:
mags = {}
diffs = {}

for detector in detectors:
    partial_data_id = {'detector': detector}
    mags[detector], diffs[detector] = get_shape_mag_diff(visits, partial_data_id)

In [ ]:
# Plot all combined measurements
all_mags = []
all_diffs = []
for detector in detectors:
    all_mags.extend(mags[detector])
    all_diffs.extend(diffs[detector])

title = '{}'.format(run, detector)
saveName = os.path.join(outdir, title + '.pdf')
plot(all_mags, all_diffs, title, saveName)

------